In [1]:
pip install aif360['all']

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 19.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
pip install fairlearn

In [3]:
# Core Python Libraries
import numpy as np
import pandas as pd

# Scikit-Learn: Preprocessing, Modeling, Evaluation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# XGBoost
from xgboost import XGBClassifier

# AIF360: Datasets and Metrics
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric

# AIF360: Preprocessing Algorithms
from aif360.algorithms.preprocessing import (
    Reweighing,
    DisparateImpactRemover,
    LFR
)

# AIF360: In-processing Algorithms
from aif360.algorithms.inprocessing import (
    MetaFairClassifier,
    GerryFairClassifier,
    PrejudiceRemover,
    ExponentiatedGradientReduction,
    GridSearchReduction,
    ARTClassifier,
    AdversarialDebiasing
)

#AIF360: Post-processing Algorithms
from aif360.algorithms.postprocessing import (
    RejectOptionClassification,
    CalibratedEqOddsPostprocessing,
    EqOddsPostprocessing
)

# TensorFlow for AdversarialDebiasing
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()


/usr/local/lib/python3.11/dist-packages/inFairness/utils/ndcg.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  vect_normalized_discounted_cumulative_gain = vmap(
/usr/local/lib/python3.11/dist-packages/inFairness/utils/ndcg.py:48: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  monte_carlo_vect_ndcg = vmap(vect_normalized_discounted

In [4]:
#data loading

In [52]:
def load_and_preprocess_taiwan_data():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls"

    # Load dataset with header at row 1
    df = pd.read_excel(url, header=1)

    # Rename target column and remap: 0 = No Default → 1 (favorable), 1 = Default → 0 (unfavorable)
    df.rename(columns={"default payment next month": "target"}, inplace=True)
    df['target'] = df['target'].map({0: 1, 1: 0})  # flip targets for fairness alignment

    # Protected attribute: age >= 25 is privileged (1), else unprivileged (0)
    df['age_binary'] = df['AGE'].apply(lambda x: 1 if x >= 25 else 0)

    # Drop ID and AGE (already used in age_binary)
    df = df.drop(columns=["ID", "AGE"])

    # One-hot encode categorical variables
    df_encoded = pd.get_dummies(df.drop(columns=['target', 'age_binary']), drop_first=True)

    X = df_encoded
    y = df['target']
    protected = df['age_binary']

    # Normalize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Train-test split
    X_train, X_test, y_train, y_test, prot_train, prot_test = train_test_split(
        X_scaled, y, protected, test_size=0.3, random_state=42, stratify=y
    )

    return X_train, X_test, y_train, y_test, prot_train, prot_test


In [53]:
def load_taiwan_credit_data():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls"

    # Read Excel file with header row at index 1
    df = pd.read_excel(url, header=1)

    # Rename and remap target column: 0 = No default (favorable), 1 = Default (unfavorable)
    df.rename(columns={"default payment next month": "target"}, inplace=True)
    df['target'] = df['target'].map({0: 1, 1: 0})

    # Protected attribute: age >= 25 -> 1 (privileged), < 25 -> 0 (unprivileged)
    df['age_binary'] = df['AGE'].apply(lambda x: 1 if x >= 25 else 0)

    # Features excluding ID, AGE, target, and protected attribute
    X_raw = df.drop(columns=["ID", "AGE", "target", "age_binary"])
    y = df['target'].values
    prot = df['age_binary'].values

    return X_raw, y, prot

def preprocess_data(X_raw, y, prot):
    cat_cols = X_raw.select_dtypes(include='object').columns.tolist()
    num_cols = X_raw.select_dtypes(exclude='object').columns.tolist()

    X_raw_train, X_raw_test, y_train, y_test, prot_train, prot_test = train_test_split(
        X_raw, y, prot, test_size=0.3, random_state=42, stratify=y
    )

    encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
    X_train_cat = encoder.fit_transform(X_raw_train[cat_cols])
    X_test_cat = encoder.transform(X_raw_test[cat_cols])

    X_train_num = X_raw_train[num_cols].values
    X_test_num = X_raw_test[num_cols].values

    X_train = np.hstack((X_train_num, X_train_cat))
    X_test = np.hstack((X_test_num, X_test_cat))

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test, prot_train, prot_test


In [54]:
def load_taiwan_credit_data_inprocess():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls"

    # Load dataset from Excel with header at row 1
    df = pd.read_excel(url, header=1)

    # Rename and remap label column
    df.rename(columns={"default payment next month": "target"}, inplace=True)
    df['target'] = df['target'].map({0: 1, 1: 0})  # No default = favorable (1), default = unfavorable (0)

    # Binary protected attribute: AGE >= 25 → privileged (1), else unprivileged (0)
    df['age_binary'] = df['AGE'].apply(lambda x: 1 if x >= 25 else 0)

    # Drop ID, AGE, target and age_binary to isolate X_raw
    X_raw = df.drop(columns=["ID", "AGE", "target", "age_binary"])
    y = df['target'].values
    prot = df['age_binary'].values

    # Train-test split
    X_train_raw, X_test_raw, y_train, y_test, prot_train, prot_test = train_test_split(
        X_raw, y, prot, test_size=0.3, random_state=42, stratify=y
    )

    # Identify categorical and numeric features
    categorical_cols = X_raw.select_dtypes(include='object').columns.tolist()
    numeric_cols = X_raw.select_dtypes(exclude='object').columns.tolist()

    # One-hot encode categorical features
    encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
    X_train_cat = encoder.fit_transform(X_train_raw[categorical_cols])
    X_test_cat = encoder.transform(X_test_raw[categorical_cols])

    # Combine numerical and categorical features
    X_train_num = X_train_raw[numeric_cols].values
    X_test_num = X_test_raw[numeric_cols].values

    X_train = np.hstack((X_train_num, X_train_cat))
    X_test = np.hstack((X_test_num, X_test_cat))

    # Normalize features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test, prot_train, prot_test


In [55]:
#Utility Functions

In [56]:
def train_baseline_model(X_train, y_train, sample_weight=None):
    model = XGBClassifier(eval_metric='logloss', use_label_encoder=False)
    model.fit(X_train, y_train, sample_weight=sample_weight)
    return model


In [57]:
def evaluate_fairness(y_true, y_pred, prot, X=None):
    df = pd.DataFrame(np.hstack((X, y_true[:, None], prot[:, None])),
                      columns=[f"x{i}" for i in range(X.shape[1])] + ['label', 'protected'])

    dataset_true = BinaryLabelDataset(df=df,
                                      label_names=["label"],
                                      protected_attribute_names=["protected"],
                                      favorable_label=1, unfavorable_label=0)

    pred_dataset = dataset_true.copy()
    pred_dataset.labels = y_pred.reshape(-1, 1)

    metric = ClassificationMetric(dataset_true, pred_dataset,
                                  privileged_groups=[{'protected': 1}],
                                  unprivileged_groups=[{'protected': 0}])

    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'disparate_impact': metric.disparate_impact(),
        'statistical_parity_difference': metric.statistical_parity_difference(),
        'equal_opportunity_difference': metric.equal_opportunity_difference()
    }


In [58]:
def evaluate_model(model, X_test, y_test, prot_test):
    y_pred = model.predict(X_test)
    return evaluate_fairness(y_test, y_pred, prot_test, X_test)


In [59]:
#XGboost
X_train, X_test, y_train, y_test, prot_train, prot_test = load_and_preprocess_taiwan_data()
baseline_model = train_baseline_model(X_train, y_train)
baseline_metrics = evaluate_model(baseline_model, X_test, y_test.to_numpy(), prot_test.to_numpy())

baseline_metrics

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [00:04:17] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


{'accuracy': 0.8147777777777778,
 'disparate_impact': np.float64(0.9344756978122642),
 'statistical_parity_difference': np.float64(-0.057739791073124436),
 'equal_opportunity_difference': np.float64(-0.01436395582737049)}

In [60]:
inprocess_results = {}

In [61]:
##disparate impact

In [62]:
def apply_disparate_impact_remover(X_train, y_train, prot_train, repair_level=1.0):
    df = pd.DataFrame(X_train)
    df['target'] = y_train.values
    df['protected'] = prot_train.values

    dataset = BinaryLabelDataset(
        favorable_label=1,
        unfavorable_label=0,
        df=df,
        label_names=['target'],
        protected_attribute_names=['protected']
    )

    dir_remover = DisparateImpactRemover(repair_level=repair_level)
    repaired_dataset = dir_remover.fit_transform(dataset)

    X_repaired = pd.DataFrame(repaired_dataset.features)
    y_repaired = pd.Series(repaired_dataset.labels.ravel())
    prot_repaired = pd.Series(repaired_dataset.protected_attributes.ravel())

    return X_repaired, y_repaired, prot_repaired


In [63]:
# Apply Disparate Impact Remover to both train and test sets
X_train_repaired, y_train_repaired, prot_train_repaired = apply_disparate_impact_remover(
    pd.DataFrame(X_train), y_train, prot_train
)

X_test_repaired, y_test_repaired, prot_test_repaired = apply_disparate_impact_remover(
    pd.DataFrame(X_test), y_test, prot_test
)

# Train on repaired training data
fair_model = train_baseline_model(X_train_repaired, y_train_repaired)

# Evaluate on repaired test data
fair_metrics = evaluate_model(
    fair_model,
    X_test_repaired.to_numpy(),
    y_test_repaired.to_numpy(),
    prot_test_repaired.to_numpy()
)
inprocess_results['Disparate_Impact_Remover']=fair_metrics
fair_metrics


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [00:04:28] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


{'accuracy': 0.8094444444444444,
 'disparate_impact': np.float64(0.9334450027917365),
 'statistical_parity_difference': np.float64(-0.056600189933523226),
 'equal_opportunity_difference': np.float64(-0.013857416296440661)}

In [64]:
##LFR

In [65]:
def apply_lfr(X_train, y_train, prot_train):
    df = pd.DataFrame(X_train)
    df['target'] = y_train.values
    df['protected'] = prot_train.values

    dataset = BinaryLabelDataset(
        favorable_label=1,
        unfavorable_label=0,
        df=df,
        label_names=['target'],
        protected_attribute_names=['protected']
    )

    lfr = LFR(unprivileged_groups=[{'protected': 0}],
          privileged_groups=[{'protected': 1}],
          k=10, Ax=0.01, Ay=1.0, Az=0.1, verbose=0)


    lfr.fit(dataset)
    transformed_dataset = lfr.transform(dataset)

    X_transformed = pd.DataFrame(transformed_dataset.features)
    y_transformed = pd.Series(transformed_dataset.labels.ravel())
    prot_transformed = pd.Series(transformed_dataset.protected_attributes.ravel())

    return X_transformed, y_transformed, prot_transformed, lfr


In [66]:
X_train_lfr, y_train_lfr, prot_train_lfr, lfr_model = apply_lfr(pd.DataFrame(X_train), y_train, prot_train)


In [67]:
print("Unique labels in y_train_lfr:", np.unique(y_train_lfr))


Unique labels in y_train_lfr: [0. 1.]


In [68]:
fair_lfr_model = train_baseline_model(X_train_lfr, y_train_lfr)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [00:05:51] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [69]:
# Apply LFR transformation on test set
df_test = pd.DataFrame(X_test)
df_test['target'] = y_test.values
df_test['protected'] = prot_test.values

test_dataset = BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=df_test,
    label_names=['target'],
    protected_attribute_names=['protected']
)

transformed_test_dataset = lfr_model.transform(test_dataset)

X_test_lfr = pd.DataFrame(transformed_test_dataset.features)
y_test_lfr = pd.Series(transformed_test_dataset.labels.ravel())
prot_test_lfr = pd.Series(transformed_test_dataset.protected_attributes.ravel())

# Evaluate
fair_lfr_metrics = evaluate_model(
    fair_lfr_model,
    X_test_lfr.to_numpy(),
    y_test_lfr.to_numpy(),
    prot_test_lfr.to_numpy()
)
inprocess_results['LFR']=fair_lfr_metrics

fair_lfr_metrics


{'accuracy': 0.9992222222222222,
 'disparate_impact': np.float64(0.9374173845662974),
 'statistical_parity_difference': np.float64(-0.05588115588115594),
 'equal_opportunity_difference': np.float64(-0.0009281793974816877)}

In [70]:
#reweighing

In [71]:
def apply_reweighing(X_train, y_train, prot_train):
    df_train = pd.DataFrame(np.hstack((X_train, y_train.reshape(-1, 1), prot_train.reshape(-1, 1))),
                            columns=[f"x{i}" for i in range(X_train.shape[1])] + ["label", "protected"])
    bld_train = BinaryLabelDataset(
        favorable_label=1,
        unfavorable_label=0,
        df=df_train,
        label_names=["label"],
        protected_attribute_names=["protected"]
    )
    RW = Reweighing(unprivileged_groups=[{'protected': 0}], privileged_groups=[{'protected': 1}])
    bld_rw = RW.fit_transform(bld_train)

    return bld_rw.features, bld_rw.labels.ravel(), bld_rw.instance_weights


In [72]:
X_train, X_test, y_train, y_test, prot_train, prot_test = preprocess_data(*load_taiwan_credit_data())

X_rw, y_rw, sample_weights = apply_reweighing(X_train, y_train, prot_train)

expected_num_features = X_rw.shape[1]

if X_test.shape[1] < expected_num_features:
    padding = expected_num_features - X_test.shape[1]
    X_test_aligned = np.hstack((X_test, np.zeros((X_test.shape[0], padding))))
elif X_test.shape[1] > expected_num_features:
    X_test_aligned = X_test[:, :expected_num_features]
else:
    X_test_aligned = X_test

model_rw = train_baseline_model(X_rw, y_rw, sample_weight=sample_weights)

reweighing_metric = evaluate_model(model_rw, X_test_aligned, y_test, prot_test)

inprocess_results['reweighing'] = reweighing_metric
reweighing_metric


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [00:05:58] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


{'accuracy': 0.8106666666666666,
 'disparate_impact': np.float64(0.9421403668334091),
 'statistical_parity_difference': np.float64(-0.05178401845068514),
 'equal_opportunity_difference': np.float64(-0.012024560805048656)}

In [73]:
for model, metrics in inprocess_results.items():
    print(f"🔹 {model}")
    for k, v in metrics.items():
        print(f"  {k}: {v:.4f}")
    print()


🔹 Disparate_Impact_Remover
  accuracy: 0.8094
  disparate_impact: 0.9334
  statistical_parity_difference: -0.0566
  equal_opportunity_difference: -0.0139

🔹 LFR
  accuracy: 0.9992
  disparate_impact: 0.9374
  statistical_parity_difference: -0.0559
  equal_opportunity_difference: -0.0009

🔹 reweighing
  accuracy: 0.8107
  disparate_impact: 0.9421
  statistical_parity_difference: -0.0518
  equal_opportunity_difference: -0.0120



In [74]:
#Inprocess

In [75]:
X_train, X_test, y_train, y_test, prot_train, prot_test = load_taiwan_credit_data_inprocess()
results = {}

In [76]:
def train_gerryfair(X_train, y_train, prot_train, X_test, y_test, prot_test):
    df_train = pd.DataFrame(np.hstack((X_train, y_train[:, None], prot_train[:, None])),
                            columns=[f"x{i}" for i in range(X_train.shape[1])] + ["label", "protected"])
    bld_train = BinaryLabelDataset(df=df_train, label_names=["label"], protected_attribute_names=["protected"],
                                   favorable_label=1, unfavorable_label=0)

    df_test = pd.DataFrame(np.hstack((X_test, y_test[:, None], prot_test[:, None])),
                           columns=[f"x{i}" for i in range(X_test.shape[1])] + ["label", "protected"])
    bld_test = BinaryLabelDataset(df=df_test, label_names=["label"], protected_attribute_names=["protected"],
                                  favorable_label=1, unfavorable_label=0)

    clf = GerryFairClassifier(C=100, printflag=False, gamma=0.005, fairness_def='FP', max_iters=50)
    clf.fit(bld_train)
    pred = clf.predict(bld_test)

    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)


In [77]:
results['GerryFair'] = train_gerryfair(X_train, y_train, prot_train, X_test, y_test, prot_test)


In [78]:
def train_prejudice_remover(X_train, y_train, prot_train, X_test, y_test, prot_test):
    df_train = pd.DataFrame(np.hstack((X_train, y_train.reshape(-1, 1), prot_train.reshape(-1, 1))),
                            columns=[f"x{i}" for i in range(X_train.shape[1])] + ["label", "protected"])
    bld_train = BinaryLabelDataset(df=df_train,
                                   label_names=["label"],
                                   protected_attribute_names=["protected"],
                                   favorable_label=1,
                                   unfavorable_label=0)

    df_test = pd.DataFrame(np.hstack((X_test, y_test.reshape(-1, 1), prot_test.reshape(-1, 1))),
                           columns=[f"x{i}" for i in range(X_test.shape[1])] + ["label", "protected"])
    bld_test = BinaryLabelDataset(df=df_test,
                                  label_names=["label"],
                                  protected_attribute_names=["protected"],
                                  favorable_label=1,
                                  unfavorable_label=0)

    clf = PrejudiceRemover(sensitive_attr="protected", eta=25.0)
    clf.fit(bld_train)
    pred = clf.predict(bld_test)

    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)


In [79]:
results['PrejudiceRemover'] = train_prejudice_remover(X_train, y_train, prot_train, X_test, y_test, prot_test)


In [80]:
def train_expgrad(X_train, y_train, prot_train, X_test, y_test, prot_test):
    df_train = pd.DataFrame(np.hstack((X_train, y_train[:, None], prot_train[:, None])),
                            columns=[f"x{i}" for i in range(X_train.shape[1])] + ["label", "protected"])
    bld_train = BinaryLabelDataset(df=df_train,
                                   label_names=["label"],
                                   protected_attribute_names=["protected"],
                                   favorable_label=1,
                                   unfavorable_label=0)

    df_test = pd.DataFrame(np.hstack((X_test, y_test[:, None], prot_test[:, None])),
                           columns=[f"x{i}" for i in range(X_test.shape[1])] + ["label", "protected"])
    bld_test = BinaryLabelDataset(df=df_test,
                                  label_names=["label"],
                                  protected_attribute_names=["protected"],
                                  favorable_label=1,
                                  unfavorable_label=0)

    expgrad = ExponentiatedGradientReduction(
        estimator=LogisticRegression(solver='liblinear'),
        constraints="DemographicParity"
    )
    expgrad.fit(bld_train)

    pred = expgrad.predict(bld_test)
    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)


In [81]:
results['ExponentiatedGradient'] = train_expgrad(X_train, y_train, prot_train, X_test, y_test, prot_test)


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [82]:
def train_gridsearch(X_train, y_train, prot_train, X_test, y_test, prot_test):
    df_train = pd.DataFrame(np.hstack((X_train, y_train[:, None], prot_train[:, None])),
                            columns=[f"x{i}" for i in range(X_train.shape[1])] + ["label", "protected"])
    bld_train = BinaryLabelDataset(df=df_train,
                                   label_names=["label"],
                                   protected_attribute_names=["protected"],
                                   favorable_label=1,
                                   unfavorable_label=0)

    df_test = pd.DataFrame(np.hstack((X_test, y_test[:, None], prot_test[:, None])),
                           columns=[f"x{i}" for i in range(X_test.shape[1])] + ["label", "protected"])
    bld_test = BinaryLabelDataset(df=df_test,
                                  label_names=["label"],
                                  protected_attribute_names=["protected"],
                                  favorable_label=1,
                                  unfavorable_label=0)

    grid = GridSearchReduction(
        estimator=LogisticRegression(solver='liblinear'),
        constraints="DemographicParity"
    )
    grid.fit(bld_train)

    pred = grid.predict(bld_test)
    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)


In [83]:
results['GridSearch'] = train_gridsearch(X_train, y_train, prot_train, X_test, y_test, prot_test)

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [84]:
tf.disable_eager_execution()  # Required for TF1 compatibility in AIF360

def train_adversarial_debiasing(X_train, y_train, prot_train, X_test, y_test, prot_test):
    # Train data
    tf.reset_default_graph()
    df_train = pd.DataFrame(np.hstack((X_train, y_train.reshape(-1, 1), prot_train.reshape(-1, 1))),
                            columns=[f"x{i}" for i in range(X_train.shape[1])] + ["label", "protected"])
    bld_train = BinaryLabelDataset(df=df_train,
                                   label_names=["label"],
                                   protected_attribute_names=["protected"],
                                   favorable_label=1,
                                   unfavorable_label=0)

    # Test data
    df_test = pd.DataFrame(np.hstack((X_test, y_test.reshape(-1, 1), prot_test.reshape(-1, 1))),
                           columns=[f"x{i}" for i in range(X_test.shape[1])] + ["label", "protected"])
    bld_test = BinaryLabelDataset(df=df_test,
                                  label_names=["label"],
                                  protected_attribute_names=["protected"],
                                  favorable_label=1,
                                  unfavorable_label=0)

    # TensorFlow session
    sess = tf.Session()

    clf = AdversarialDebiasing(
        privileged_groups=[{'protected': 1}],
        unprivileged_groups=[{'protected': 0}],
        scope_name='adv_debiasing',
        sess=sess,
        num_epochs=50,
        batch_size=64,
        debias=True
    )

    clf.fit(bld_train)
    pred = clf.predict(bld_test)

    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)


In [85]:
results['AdversarialDebiasing'] = train_adversarial_debiasing(X_train, y_train, prot_train, X_test, y_test, prot_test)


epoch 0; iter: 0; batch classifier loss: 0.631093; batch adversarial loss: 0.516974
epoch 0; iter: 200; batch classifier loss: 0.628598; batch adversarial loss: 0.386894
epoch 1; iter: 0; batch classifier loss: 0.458456; batch adversarial loss: 0.453721
epoch 1; iter: 200; batch classifier loss: 0.549874; batch adversarial loss: 0.331929
epoch 2; iter: 0; batch classifier loss: 0.496895; batch adversarial loss: 0.401980
epoch 2; iter: 200; batch classifier loss: 0.410916; batch adversarial loss: 0.492252
epoch 3; iter: 0; batch classifier loss: 0.481046; batch adversarial loss: 0.377303
epoch 3; iter: 200; batch classifier loss: 0.471168; batch adversarial loss: 0.246060
epoch 4; iter: 0; batch classifier loss: 0.438584; batch adversarial loss: 0.224712
epoch 4; iter: 200; batch classifier loss: 0.455554; batch adversarial loss: 0.221404
epoch 5; iter: 0; batch classifier loss: 0.582365; batch adversarial loss: 0.260500
epoch 5; iter: 200; batch classifier loss: 0.362993; batch adversa

In [86]:
print("Fairness Evaluation Across Compatible AIF360 In-processing Algorithms\n")
for model, metrics in results.items():
    print(f"🔹 {model}")
    for metric_name, value in metrics.items():
        print(f"  {metric_name}: {value:.4f}")
    print()


Fairness Evaluation Across Compatible AIF360 In-processing Algorithms

🔹 GerryFair
  accuracy: 0.8012
  disparate_impact: 0.9767
  statistical_parity_difference: -0.0223
  equal_opportunity_difference: -0.0069

🔹 PrejudiceRemover
  accuracy: 0.8099
  disparate_impact: 0.9902
  statistical_parity_difference: -0.0091
  equal_opportunity_difference: 0.0013

🔹 ExponentiatedGradient
  accuracy: 0.8108
  disparate_impact: 0.9641
  statistical_parity_difference: -0.0336
  equal_opportunity_difference: -0.0088

🔹 GridSearch
  accuracy: 0.8099
  disparate_impact: 0.9801
  statistical_parity_difference: -0.0186
  equal_opportunity_difference: -0.0023

🔹 AdversarialDebiasing
  accuracy: 0.8214
  disparate_impact: 0.9310
  statistical_parity_difference: -0.0619
  equal_opportunity_difference: -0.0218



In [87]:
#postprocessing

In [88]:
X_train, X_test, y_train, y_test, prot_train, prot_test = load_taiwan_credit_data_inprocess()
post_results = {}

In [91]:
def train_roc_postprocessing_with_xgb(X_train, y_train, prot_train, X_test, y_test, prot_test):
    model = train_baseline_model(X_train, y_train)

    y_prob = model.predict_proba(X_test)[:, 1]
    y_pred = model.predict(X_test)

    bld_test = BinaryLabelDataset(
        favorable_label=1,
        unfavorable_label=0,
        df=pd.DataFrame(np.hstack((X_test, y_test[:, None], prot_test[:, None])),
                        columns=[f'x{i}' for i in range(X_test.shape[1])] + ['label', 'protected']),
        label_names=['label'],
        protected_attribute_names=['protected']
    )

    bld_pred = bld_test.copy()
    bld_pred.scores = y_prob.reshape(-1, 1)
    bld_pred.labels = y_pred.reshape(-1, 1)

    roc = RejectOptionClassification(
        unprivileged_groups=[{'protected': 0}],
        privileged_groups=[{'protected': 1}],
        low_class_thresh=0.01, high_class_thresh=0.99,
        num_class_thresh=100, num_ROC_margin=50,
        metric_name="Statistical parity difference",
        metric_ub=0.05, metric_lb=-0.05
    )
    roc = roc.fit(bld_test, bld_pred)
    pred = roc.predict(bld_pred)

    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)


In [93]:
post_results['RejectOptionClassification'] = train_roc_postprocessing_with_xgb(X_train, y_train, prot_train, X_test, y_test, prot_test)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [00:08:39] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [96]:
def train_calibrated_eq_odds_with_xgb(X_train, y_train, prot_train, X_test, y_test, prot_test):
    model = train_baseline_model(X_train, y_train)

    y_prob = model.predict_proba(X_test)[:, 1]

    bld_test = BinaryLabelDataset(
        favorable_label=1,
        unfavorable_label=0,
        df=pd.DataFrame(np.hstack((X_test, y_test[:, None], prot_test[:, None])),
                        columns=[f'x{i}' for i in range(X_test.shape[1])] + ['label', 'protected']),
        label_names=['label'],
        protected_attribute_names=['protected']
    )

    bld_pred = bld_test.copy()
    bld_pred.scores = y_prob.reshape(-1, 1)

    ceo = CalibratedEqOddsPostprocessing(
        privileged_groups=[{'protected': 1}],
        unprivileged_groups=[{'protected': 0}],
        cost_constraint="fnr",
        seed=42
    )
    ceo = ceo.fit(bld_test, bld_pred)
    pred = ceo.predict(bld_pred)

    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)


In [97]:
post_results['CalibratedEqOdds'] = train_calibrated_eq_odds_with_xgb(X_train, y_train, prot_train, X_test, y_test, prot_test)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [00:09:21] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [98]:
def train_equalized_odds_with_xgb(X_train, y_train, prot_train, X_test, y_test, prot_test):
    model = train_baseline_model(X_train, y_train)

    y_pred = model.predict(X_test)

    bld_test = BinaryLabelDataset(
        favorable_label=1,
        unfavorable_label=0,
        df=pd.DataFrame(np.hstack((X_test, y_test[:, None], prot_test[:, None])),
                        columns=[f'x{i}' for i in range(X_test.shape[1])] + ['label', 'protected']),
        label_names=['label'],
        protected_attribute_names=['protected']
    )

    bld_pred = bld_test.copy()
    bld_pred.labels = y_pred.reshape(-1, 1)

    eq = EqOddsPostprocessing(
        privileged_groups=[{'protected': 1}],
        unprivileged_groups=[{'protected': 0}]
    )
    eq = eq.fit(bld_test, bld_pred)
    pred = eq.predict(bld_pred)

    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)


In [99]:
post_results['EqualizedOdds'] = train_equalized_odds_with_xgb(X_train, y_train, prot_train, X_test, y_test, prot_test)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [00:09:23] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [100]:
for model, metrics in post_results.items():
    print(f"🔹 {model}")
    for k, v in metrics.items():
        print(f"  {k}: {v:.4f}")
    print()


🔹 RejectOptionClassification
  accuracy: 0.7416
  disparate_impact: 0.9301
  statistical_parity_difference: -0.0481
  equal_opportunity_difference: -0.0114

🔹 CalibratedEqOdds
  accuracy: 0.7981
  disparate_impact: 0.8616
  statistical_parity_difference: -0.1322
  equal_opportunity_difference: -0.0501

🔹 EqualizedOdds
  accuracy: 0.8129
  disparate_impact: 0.9731
  statistical_parity_difference: -0.0237
  equal_opportunity_difference: -0.0014

